### Imports

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
import fastai

In [4]:
from fastai.imports import *
# from fastai.structured import *

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [7]:
pwd

'/home/ademyanchuk/Projects/Kaggle/kaggle_google_revenue/notebooks'

In [9]:
import sys
sys.path.append('../src')

In [10]:
from workflow.data_utils import load_csv, train_valid_split

### Constants

In [7]:
DATE_COLUMN ='date'
SPLIT_DATE ='2017-05-01'
ID_COLUMN = 'fullVisitorId'
TARGET_COLUMN = 'totals_transactionRevenue'

### Loads 

In [8]:
%%time
full_train = load_csv('../data/processed_data/processed_train.csv')
test_df = load_csv('../data/processed_data/processed_test.csv')

In [9]:
full_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 35 columns):
channelGrouping                                 903653 non-null object
date                                            903653 non-null int64
fullVisitorId                                   903653 non-null object
sessionId                                       903653 non-null object
visitId                                         903653 non-null int64
visitNumber                                     903653 non-null int64
visitStartTime                                  903653 non-null int64
device_browser                                  903653 non-null object
device_deviceCategory                           903653 non-null object
device_isMobile                                 903653 non-null bool
device_operatingSystem                          903653 non-null object
geoNetwork_city                                 903653 non-null object
geoNetwork_continent                       

In [10]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804684 entries, 0 to 804683
Data columns (total 34 columns):
channelGrouping                                 804684 non-null object
date                                            804684 non-null int64
fullVisitorId                                   804684 non-null object
sessionId                                       804684 non-null object
visitId                                         804684 non-null int64
visitNumber                                     804684 non-null int64
visitStartTime                                  804684 non-null int64
device_browser                                  804684 non-null object
device_deviceCategory                           804684 non-null object
device_isMobile                                 804684 non-null bool
device_operatingSystem                          804684 non-null object
geoNetwork_city                                 804684 non-null object
geoNetwork_continent                       

In [12]:
all(full_train.drop(TARGET_COLUMN, axis=1).columns == test_df.columns)

True

### Workflow

In [15]:
full_train[TARGET_COLUMN] = full_train[TARGET_COLUMN].fillna(value=0)

In [16]:
%%time
train_df, valid_df = train_valid_split(
    full_train, id_column=ID_COLUMN, time_column=DATE_COLUMN, split_date=SPLIT_DATE)

In [17]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 711463 entries, 0 to 711462
Data columns (total 35 columns):
channelGrouping                                 711463 non-null object
date                                            711463 non-null datetime64[ns]
fullVisitorId                                   711463 non-null object
sessionId                                       711463 non-null object
visitId                                         711463 non-null int64
visitNumber                                     711463 non-null int64
visitStartTime                                  711463 non-null int64
device_browser                                  711463 non-null object
device_deviceCategory                           711463 non-null object
device_isMobile                                 711463 non-null bool
device_operatingSystem                          711463 non-null object
geoNetwork_city                                 711463 non-null object
geoNetwork_continent              

In [18]:
valid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192190 entries, 0 to 192189
Data columns (total 35 columns):
channelGrouping                                 192190 non-null object
date                                            192190 non-null datetime64[ns]
fullVisitorId                                   192190 non-null object
sessionId                                       192190 non-null object
visitId                                         192190 non-null int64
visitNumber                                     192190 non-null int64
visitStartTime                                  192190 non-null int64
device_browser                                  192190 non-null object
device_deviceCategory                           192190 non-null object
device_isMobile                                 192190 non-null bool
device_operatingSystem                          192190 non-null object
geoNetwork_city                                 192190 non-null object
geoNetwork_continent              

#### Predicting mean of train

In [19]:
grouped_train = train_df.groupby(ID_COLUMN, as_index=False).agg({TARGET_COLUMN: sum})

In [22]:
grouped_train['log_revenue'] = np.log1p(grouped_train.totals_transactionRevenue)

In [33]:
train_log_target_mean = grouped_train.log_revenue.mean()

In [26]:
grouped_valid = valid_df.groupby(ID_COLUMN, as_index=False).agg({TARGET_COLUMN: sum})

In [30]:
grouped_valid['log_revenue'] = np.log1p(grouped_valid.totals_transactionRevenue)

In [34]:
grouped_valid['predicted'] = train_log_target_mean

In [35]:
grouped_valid

,fullVisitorId,totals_transactionRevenue,log_revenue,predicted
0,0000062267706107999,0.0,0.0,0.241303
1,0000085059828173212,0.0,0.0,0.241303
2,0000197671390269035,0.0,0.0,0.241303
3,000026722803385797,0.0,0.0,0.241303
4,0000436683523507380,0.0,0.0,0.241303
5,0000450371054833295,0.0,0.0,0.241303
6,0000458812883559498,0.0,0.0,0.241303
7,0000572434142265465,0.0,0.0,0.241303
8,0000618629079057366,0.0,0.0,0.241303
9,0000650183049109374,0.0,0.0,0.241303


In [38]:
mse = mean_squared_error(grouped_valid.log_revenue, grouped_valid.predicted)

In [39]:
rmse = np.sqrt(mse)

In [40]:
rmse

2.2117682568222303

In [41]:
submit = pd.read_csv('../data/sample_submission.csv')

In [43]:
submit.PredictedLogRevenue = train_log_target_mean

In [45]:
submit.to_csv('../submissions/train_mean.csv', header=True, index=False)